# 評価：音楽属性値の類似度

In [ ]:
import numpy as np
import pandas as pd
import torch as th

In [ ]:
# コサイン類似度
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:

# 閾値
thresholds = np.squeeze(np.load("../../../../../../data/threshold.npy", allow_pickle=True)) #閾値
# 音楽属性の番号
feature_index = np.load("../../../../../../data/feature_index.npy", allow_pickle=True) #特徴量100
# 100つの音楽属性値を揃える重み:σ
thresholds_weight0_100 = th.load('./std_mean.pt') #100
thresholds_weight_100 = th.where(thresholds_weight0_100[0,:] == 0.0, (th.tensor(0.0).float()), ((1.0 / thresholds_weight0_100[0,:]).float())).to('cpu').detach().numpy().copy()
# 18*2つの音楽属性値を揃える重み:σ
thresholds_weight0_18 = th.load('./std_mean_train_a18_2.pt')#)./std_mean.pt') 
thresholds_weight_18 = th.where(thresholds_weight0_18[0,:] == 0.0, (th.tensor(0.0).float()), ((1.0 / thresholds_weight0_18[0,:]).float())).to('cpu').detach().numpy().copy() 

mask18_2 = np.zeros((1,100))
mask18_V = np.zeros((1,100))
mask18_A = np.zeros((1,100))
number_a18_2 = [1, 2, 3, 5, 20, 22, 25, 26, 35, 37, 40, 41, 42, 43, 44, 45, 49, 57, 62, 63, 64, 67, 68, 69, 70, 71, 73, 75, 77, 81, 84, 85, 87, 92, 95, 99] # 音楽属性値18*2
mask18_2[:,number_a18_2] = 1.0 #100→36
mask18_V[:,number_a18_2[:18]] = 1.0 # Valence (18)
mask18_A[:,number_a18_2[18:]] = 1.0 # Arousal (18)

### 生成音楽データ

* Classifier Guidance Diffusion Model (CG)の場合

In [ ]:
# csvファイルの中身は実際の音楽属性値：閾値処理が必要
# jSymbolicで取得
df1_Q1_4_CF = pd.read_csv('.csv',usecols=feature_index+1)
threshold_Q1_4_CF = np.zeros((0,100))
threshold_Q1_4_CF_18Valence_noweight = np.zeros((0,100))
threshold_Q1_4_CF_18Arousal_noweight = np.zeros((0,100))
threshold_Q1_4_CF_18_noweight = np.zeros((0,100))
threshold_Q1_4_CF_18Valence = np.zeros((0,100))
threshold_Q1_4_CF_18Arousal = np.zeros((0,100))
threshold_Q1_4_CF_18 = np.zeros((0,100))
#print(df1)
for i in range(16):
    # csvからデータ取得
    a_Q1_4_CF0 = np.squeeze(np.array(df1_Q1_4_CF[i:i+1].values))
    a_Q1_4_CF = [np.float64(a_Q1_4_CF00) for a_Q1_4_CF00 in a_Q1_4_CF0] 
    # 閾値処理
    threshold_Q1_4_CF = np.append(threshold_Q1_4_CF, (a_Q1_4_CF - thresholds)*thresholds_weight_100) # ① 100
    threshold_Q1_4_CF_18Valence_noweight = np.append(threshold_Q1_4_CF_18Valence_noweight , (a_Q1_4_CF - thresholds)*mask18_V) # ② Valence：18*2  σなし
    threshold_Q1_4_CF_18Arousal_noweight = np.append(threshold_Q1_4_CF_18Arousal_noweight, (a_Q1_4_CF - thresholds)*mask18_A) # ③ Arousal：18*2 σなし
    threshold_Q1_4_CF_18_noweight = np.append(threshold_Q1_4_CF_18_noweight, (a_Q1_4_CF - thresholds)*mask18_2) # ④ 18*2 σなし
    threshold_Q1_4_CF_18Valence = np.append(threshold_Q1_4_CF_18Valence , (a_Q1_4_CF - thresholds)*mask18_V*thresholds_weight_18) # ⑤ Valence：18*2 σあり
    threshold_Q1_4_CF_18Arousal = np.append(threshold_Q1_4_CF_18Arousal, (a_Q1_4_CF - thresholds)*mask18_A*thresholds_weight_18) # ⑥ Arousal：18*2 σあり
    threshold_Q1_4_CF_18 = np.append(threshold_Q1_4_CF_18, (a_Q1_4_CF - thresholds)*mask18_2*thresholds_weight_18) # ⑦ 18*2 σあり

threshold_Q1_4_all_CF = threshold_Q1_4_CF.reshape(16,100)
threshold_Q1_4_all_CF_18Valence_noweight = threshold_Q1_4_CF_18Valence_noweight.reshape(16,100)
threshold_Q1_4_all_CF_18Arousal_noweight = threshold_Q1_4_CF_18Arousal_noweight.reshape(16,100)
threshold_Q1_4_all_CF_18_noweight = threshold_Q1_4_CF_18_noweight.reshape(16,100)
threshold_Q1_4_all_CF_18Valence = threshold_Q1_4_CF_18Valence.reshape(16,100)
threshold_Q1_4_all_CF_18Arousal = threshold_Q1_4_CF_18Arousal.reshape(16,100)
threshold_Q1_4_all_CF_18 = threshold_Q1_4_CF_18.reshape(16,100)
print('threshold_Q1_4 = ' + str(threshold_Q1_4_all_CF))

* Classifier Free Guidance Diffusion Model (CFG)の場合

In [ ]:
df1_Q1_4_CFG = pd.read_csv('.csv',usecols=feature_index+1) #skiprows=1) 
#print(df1_CFG)
threshold_Q1_4_CFG = np.zeros((0,100))
threshold_Q1_4_CFG_18Valence_noweight = np.zeros((0,100))
threshold_Q1_4_CFG_18Arousal_noweight = np.zeros((0,100))
threshold_Q1_4_CFG_18_noweight = np.zeros((0,100))
threshold_Q1_4_CFG_18Valence = np.zeros((0,100))
threshold_Q1_4_CFG_18Arousal = np.zeros((0,100))
threshold_Q1_4_CFG_18 = np.zeros((0,100))

for j in range(8):
    # csvからデータ取得
    a_Q1_4_CFG0 = np.squeeze(np.array(df1_Q1_4_CFG[j:j+1].values))
    a_Q1_4_CFG = [np.float64(a_Q1_4_CFG00) for a_Q1_4_CFG00 in a_Q1_4_CFG0]
    # 閾値処理
    threshold_Q1_4_CFG = np.append(threshold_Q1_4_CFG, (a_Q1_4_CFG - thresholds)*thresholds_weight_100) # ① 100
    threshold_Q1_4_CFG_18Valence_noweight = np.append(threshold_Q1_4_CFG_18Valence_noweight , (a_Q1_4_CFG - thresholds)*mask18_V) # ② Valence：18*2重みなし
    threshold_Q1_4_CFG_18Arousal_noweight = np.append(threshold_Q1_4_CFG_18Arousal_noweight, (a_Q1_4_CFG - thresholds)*mask18_A) # ③ Arousal：18*2重みなし
    threshold_Q1_4_CFG_18_noweight = np.append(threshold_Q1_4_CFG_18_noweight, (a_Q1_4_CFG - thresholds)*mask18_2) # ④ 18*2の重みなし
    threshold_Q1_4_CFG_18Valence = np.append(threshold_Q1_4_CFG_18Valence , (a_Q1_4_CFG - thresholds)*mask18_V*thresholds_weight_18) # ⑤ Valence：18*2重みあり
    threshold_Q1_4_CFG_18Arousal = np.append(threshold_Q1_4_CFG_18Arousal, (a_Q1_4_CFG - thresholds)*mask18_A*thresholds_weight_18) # ⑥ Arousal：18*2重みあり
    threshold_Q1_4_CFG_18 = np.append(threshold_Q1_4_CFG_18, (a_Q1_4_CFG - thresholds)*mask18_2*thresholds_weight_18) # ⑦ 18*2重みあり
    
threshold_Q1_4_all_CFG = threshold_Q1_4_CFG.reshape(8,100)
threshold_Q1_4_all_CFG_18Valence_noweight = threshold_Q1_4_CFG_18Valence_noweight.reshape(8,100)
threshold_Q1_4_all_CFG_18Arousal_noweight = threshold_Q1_4_CFG_18Arousal_noweight.reshape(8,100)
threshold_Q1_4_all_CFG_18_noweight = threshold_Q1_4_CFG_18_noweight.reshape(8,100)
threshold_Q1_4_all_CFG_18Valence = threshold_Q1_4_CFG_18Valence.reshape(8,100)
threshold_Q1_4_all_CFG_18Arousal = threshold_Q1_4_CFG_18Arousal.reshape(8,100)
threshold_Q1_4_all_CFG_18 = threshold_Q1_4_CFG_18.reshape(8,100)

print('threshold_Q1_4 = ' + str(threshold_Q1_4_all_CFG))



### 入力データ (DEAMモデルを使用した場合)

In [ ]:
real_train  = th.load('input_emotion_18_2_DEAM.pt').to('cpu').detach().numpy().copy()
real_train1 = real_train[0,:]
real_train2 = real_train[1,:]
real_train3 = real_train[2,:]
real_train4 = real_train[3,:]

# 拡散モデルの学習データに寄せる重み
russell_train_weight = np.load('DEAM_train_weight.npz')['arr_0'] 

# 入力１：100
real_train1_threshold = (real_train1 - thresholds)*thresholds_weight_100 #* russell_train_weight
real_train2_threshold = (real_train1 - thresholds)*thresholds_weight_100 #* russell_train_weight
real_train3_threshold = (real_train1 - thresholds)*thresholds_weight_100  #* russell_train_weight
real_train4_threshold = (real_train1 - thresholds)*thresholds_weight_100  #* russell_train_weight

# 入力2：18_Valence_σなし
real_train1_18_noweight = (real_train1 - thresholds)*mask18_2 #* russell_train_weight
real_train2_18_noweight = (real_train2 - thresholds)*mask18_2 #* russell_train_weight
real_train3_18_noweight = (real_train3 - thresholds)*mask18_2 #* russell_train_weight
real_train4_18_noweight = (real_train4 - thresholds)*mask18_2 #* russell_train_weight
# 入力3：18_Arousal_σなし
real_train1_18Valence_noweight = (real_train1 - thresholds)*mask18_V #* russell_train_weight
real_train2_18Valence_noweight = (real_train2 - thresholds)*mask18_V #* russell_train_weight
real_train3_18Valence_noweight = (real_train3 - thresholds)*mask18_V #* russell_train_weight
real_train4_18Valence_noweight = (real_train4 - thresholds)*mask18_V #* russell_train_weight
# 入力4：18*2_σなし
real_train1_18Arousal_noweight = (real_train1 - thresholds)*mask18_A #* russell_train_weight
real_train2_18Arousal_noweight = (real_train2 - thresholds)*mask18_A #* russell_train_weight
real_train3_18Arousal_noweight = (real_train3 - thresholds)*mask18_A #* russell_train_weight
real_train4_18Arousal_noweight = (real_train4 - thresholds)*mask18_A #* russell_train_weight

# 入力5：18_Valence
real_train1_18Valence = (real_train1 - thresholds)*mask18_V*thresholds_weight_18 * russell_train_weight
real_train2_18Valence = (real_train2 - thresholds)*mask18_V*thresholds_weight_18 * russell_train_weight
real_train3_18Valence = (real_train3 - thresholds)*mask18_V*thresholds_weight_18 * russell_train_weight
real_train4_18Valence = (real_train4 - thresholds)*mask18_V*thresholds_weight_18 * russell_train_weight
# 入力6：18_Arousal
real_train1_18Arousal = (real_train1 - thresholds)*mask18_A*thresholds_weight_18 * russell_train_weight
real_train2_18Arousal = (real_train2 - thresholds)*mask18_A*thresholds_weight_18 * russell_train_weight
real_train3_18Arousal = (real_train3 - thresholds)*mask18_A*thresholds_weight_18 * russell_train_weight
real_train4_18Arousal = (real_train4 - thresholds)*mask18_A*thresholds_weight_18 * russell_train_weight
# 入力7：18*2_重みあり
real_train1_18 = (real_train1 - thresholds)*mask18_2*thresholds_weight_18 * russell_train_weight
real_train2_18 = (real_train2 - thresholds)*mask18_2*thresholds_weight_18 * russell_train_weight
real_train3_18 = (real_train3 - thresholds)*mask18_2*thresholds_weight_18 * russell_train_weight
real_train4_18 = (real_train4 - thresholds)*mask18_2*thresholds_weight_18 * russell_train_weight

### Cos類似度

In [ ]:
# CG
cos_sim_y_CF_Q1_4 = 0.0
cos_sim_y_CF_Q1_4_18Valence_noweight = 0.0
cos_sim_y_CF_Q1_4_18Arousal_noweight = 0.0
cos_sim_y_CF_Q1_4_18_noweight = 0.0
cos_sim_y_CF_Q1_4_18Valence = 0.0
cos_sim_y_CF_Q1_4_18Arousal = 0.0
cos_sim_y_CF_Q1_4_18 = 0.0
# CFG
cos_sim_y_CFG_Q1_4  = 0.0
cos_sim_y_CFG_Q1_4_18Valence_noweight = 0.0
cos_sim_y_CFG_Q1_4_18Arousal_noweight= 0.0
cos_sim_y_CFG_Q1_4_18_noweight = 0.0
cos_sim_y_CFG_Q1_4_18Valence = 0.0
cos_sim_y_CFG_Q1_4_18Arousal = 0.0
cos_sim_y_CFG_Q1_4_18 = 0.0

# Cos類似度
# Q4に該当する座標値を入力とした場合

# CG
'''
for i in range(16):
    cos_sim_y0_CF_Q1_4  = cos_sim(np.squeeze(threshold_Q1_4_all_CF[i]), np.squeeze(real_train4_threshold)) # 256→100
    # σなし
    cos_sim_y0_CF_Q1_4_18Valence_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18Valence_noweight[i]), np.squeeze(real_train4_18Valence_noweight)) 
    cos_sim_y0_CF_Q1_4_18Arousal_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18Arousal_noweight[i]), np.squeeze(real_train4_18Arousal_noweight)) 
    cos_sim_y0_CF_Q1_4_18_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18_noweight[i]), np.squeeze(real_train4_18_noweight)) 
    # σあり
    cos_sim_y0_CF_Q1_4_18Valence = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18Valence[i]), np.squeeze(real_train4_18Valence)) 
    cos_sim_y0_CF_Q1_4_18Arousal = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18Arousal[i]), np.squeeze(real_train4_18Arousal)) 
    cos_sim_y0_CF_Q1_4_18 = cos_sim(np.squeeze(threshold_Q1_4_all_CF_18[i]), np.squeeze(real_train4_18)) 
    
    if np.isnan(cos_sim_y0_CF_Q1_4) ==  False: 
        cos_sim_y_CF_Q1_4  = cos_sim_y_CF_Q1_4  + cos_sim_y0_CF_Q1_4 
        cos_sim_y_CF_Q1_4_18Valence_noweight = cos_sim_y_CF_Q1_4_18Valence_noweight + cos_sim_y0_CF_Q1_4_18Valence_noweight
        cos_sim_y_CF_Q1_4_18Arousal_noweight = cos_sim_y_CF_Q1_4_18Arousal_noweight + cos_sim_y0_CF_Q1_4_18Arousal_noweight 
        cos_sim_y_CF_Q1_4_18_noweight  = cos_sim_y_CF_Q1_4_18_noweight + cos_sim_y0_CF_Q1_4_18_noweight
        cos_sim_y_CF_Q1_4_18Valence = cos_sim_y_CF_Q1_4_18Valence + cos_sim_y0_CF_Q1_4_18Valence
        cos_sim_y_CF_Q1_4_18Arousal = cos_sim_y_CF_Q1_4_18Arousal + cos_sim_y0_CF_Q1_4_18Arousal 
        cos_sim_y_CF_Q1_4_18 = cos_sim_y_CF_Q1_4_18  + cos_sim_y0_CF_Q1_4_18
    # cos_sim_y = cos_sim_y + cos_sim(pred[i].cpu().detach().numpy(), vectors[i].cpu().detach().numpy()) # 256→30
print(cos_sim_y_CF_Q1_4 / 16.0)
print(cos_sim_y_CF_Q1_4_18Valence_noweight / 16.0)
print(cos_sim_y_CF_Q1_4_18Arousal_noweight / 16.0)
print(cos_sim_y_CF_Q1_4_18_noweight / 16.0)
print(cos_sim_y_CF_Q1_4_18Valence / 16.0)
print(cos_sim_y_CF_Q1_4_18Arousal / 16.0)
print(cos_sim_y_CF_Q1_4_18 / 16.0)

'''

# CFG
for j in range(8):
    cos_sim_y0_CFG_Q1_4 = cos_sim(np.squeeze(threshold_Q1_4_all_CFG[j]), np.squeeze(real_train1_threshold)) # 256→100
    # σなし
    cos_sim_y0_CFG_Q1_4_18Valence_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18Valence_noweight[j]), np.squeeze(real_train1_18Valence_noweight)) 
    cos_sim_y0_CFG_Q1_4_18Arousal_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18Arousal_noweight[j]), np.squeeze(real_train1_18Arousal_noweight)) 
    cos_sim_y0_CFG_Q1_4_18_noweight = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18_noweight[j]), np.squeeze(real_train1_18_noweight)) 
    # σあり
    cos_sim_y0_CFG_Q1_4_18Valence = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18Valence[j]), np.squeeze(real_train1_18Valence)) 
    cos_sim_y0_CFG_Q1_4_18Arousal = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18Arousal[j]), np.squeeze(real_train1_18Arousal)) 
    cos_sim_y0_CFG_Q1_4_18 = cos_sim(np.squeeze(threshold_Q1_4_all_CFG_18[j]), np.squeeze(real_train1_18)) 
    
    if np.isnan(cos_sim_y0_CFG_Q1_4) ==  False: 
        cos_sim_y_CFG_Q1_4 = cos_sim_y_CFG_Q1_4 + cos_sim_y0_CFG_Q1_4
        cos_sim_y_CFG_Q1_4_18Valence_noweight = cos_sim_y_CFG_Q1_4_18Valence_noweight + cos_sim_y0_CFG_Q1_4_18Valence_noweight
        cos_sim_y_CFG_Q1_4_18Arousal_noweight = cos_sim_y_CFG_Q1_4_18Arousal_noweight + cos_sim_y0_CFG_Q1_4_18Arousal_noweight 
        cos_sim_y_CFG_Q1_4_18_noweight  = cos_sim_y_CFG_Q1_4_18_noweight + cos_sim_y0_CFG_Q1_4_18_noweight
        cos_sim_y_CFG_Q1_4_18Valence = cos_sim_y_CFG_Q1_4_18Valence + cos_sim_y0_CFG_Q1_4_18Valence
        cos_sim_y_CFG_Q1_4_18Arousal = cos_sim_y_CFG_Q1_4_18Arousal + cos_sim_y0_CFG_Q1_4_18Arousal 
        cos_sim_y_CFG_Q1_4_18 = cos_sim_y_CFG_Q1_4_18  + cos_sim_y0_CFG_Q1_4_18
print(cos_sim_y_CFG_Q1_4 / 8.0)
print(cos_sim_y_CFG_Q1_4_18Valence_noweight / 8.0)
print(cos_sim_y_CFG_Q1_4_18Arousal_noweight / 8.0)
print(cos_sim_y_CFG_Q1_4_18_noweight / 8.0)
print(cos_sim_y_CFG_Q1_4_18Valence / 8.0)
print(cos_sim_y_CFG_Q1_4_18Arousal / 8.0)
print(cos_sim_y_CFG_Q1_4_18 / 8.0)
#'''